In [20]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [2]:
anime=pd.read_csv("anime.csv")
rating=pd.read_csv("rating.csv")

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
anime.shape

(12294, 7)

In [6]:
rating.shape

(7813737, 3)

In [7]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [8]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [9]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [10]:
rating.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [11]:
print(anime['type'].mode())
print(anime['genre'].mode())

0    TV
Name: type, dtype: object
0    Hentai
Name: genre, dtype: object


In [12]:
#delete anime without rating
anime= anime.dropna(subset=['rating'])
anime['type'] = anime['type'].fillna(anime['type'].dropna().mode().values[0])
anime['genre'] = anime['genre'].fillna(anime['genre'].dropna().mode().values[0])

In [13]:
anime.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [14]:
for i,j in enumerate(rating["rating"]):
    if j==-1:
        rating["rating"][i]=np.nan
anime= anime[anime['type']=='TV']

In [15]:
rated_anime = rating.merge(anime, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
rated_anime =rated_anime[['user_id', 'name', 'rating']]
rated_anime= rated_anime[rated_anime.user_id <= 7500]

In [16]:
pivot = rated_anime.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,3 Choume no Tama: Uchi no Tama Shirimasenka?,30-sai no Hoken Taiiku,91 Days,...,"Zone of the Enders: Dolores, I",Zukkoke Knight: Don De La Mancha,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
pivot_n.fillna(0, inplace=True)
pivot_n = pivot_n.T
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]
piv_sparse =sp.csr_matrix(pivot_n.values)

In [23]:
pivot_n

user_id,1,2,3,4,5,6,7,8,10,11,...,7489,7490,7491,7492,7494,7495,7496,7497,7499,7500
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,-0.179964,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07-Ghost,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gdgd Fairies 2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iDOLM@STER Xenoglossia,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s.CRY.ed,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,-0.062983,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
anime_similarity = cosine_similarity(piv_sparse)
ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [30]:
ani_sim_df.head()
ani_sim_df.to_csv('anime_similarity.csv', index=True)

In [38]:

ani_sim_df = pd.read_csv('anime_similarity.csv', index_col=0)
anime_list = ani_sim_df.columns.values.tolist()
animeScores = [(x, 0) for x in anime_list]


In [39]:
def anime_recommendation(anime_names,animeScores):  
    for anime_name in anime_names:
        i=0
        for score in ani_sim_df[anime_name]:
            animeScores[i]=(animeScores[i][0],animeScores[i][1]+score)
            i+=1    

In [42]:
def list_recommendation(animeScores):
    sorted_animeScores = sorted(animeScores, key=lambda x: x[1], reverse=True)
    for i in range(20):
        print(sorted_animeScores[i][0])

In [43]:
anime_recommendation(["Monster","Death Parade"],animeScores)
list_recommendation(animeScores)

Death Parade
Monster
Kiseijuu: Sei no Kakuritsu
Zankyou no Terror
Boku dake ga Inai Machi
One Punch Man
Psycho-Pass
Steins;Gate
Shokugeki no Souma
No Game No Life
Fate/stay night: Unlimited Blade Works
Fate/Zero
Gekkan Shoujo Nozaki-kun
Barakamon
Noragami Aragoto
Durarara!!
Tokyo Ghoul
Ansatsu Kyoushitsu (TV)
Code Geass: Hangyaku no Lelouch
Fate/Zero 2nd Season
